In [26]:
import pandas as pd
import boto3
from helpers.aws import pull_files_s3, get_s3_client
import os
from datetime import timedelta, datetime
import concurrent.futures
import pandas_market_calendars as mcal

In [27]:
from helpers.constants import FULL_SYM, TRADING_SYMBOLS, WEEKLY_EXP

TRADING_SYMBOLS[46:92]

diff = list(set(WEEKLY_EXP) - set(TRADING_SYMBOLS))
print(diff)
diff2 = list(set(TRADING_SYMBOLS) - set(WEEKLY_EXP))
print(diff2)


['COP', 'PNC', 'LEN', 'DHR', 'IVV', 'PHM', 'ADI', 'VRTX', 'ADP', 'SCHW', 'CLX', 'LVS', 'ADM', 'FTNT', 'HSY', 'PCG', 'WMB', 'ILMN', 'CNC', 'LLY', 'KMB', 'AIG', 'KMX', 'CMA', 'TMO', 'VFC', 'NOC', 'ROST', 'COF', 'KEY', 'ELV', 'TTWO', 'ABT', 'EBAY', 'FSLR', 'TSCO', 'MRO', 'ZION', 'EA', 'HUM', 'DFS', 'HAL', 'PPG', 'CSX', 'WYNN', 'SLB', 'AAP', 'BAX', 'MPC', 'LUV', 'HON', 'TJX', 'TROW', 'GLW', 'FCX', 'TSN', 'BIIB', 'FITB', 'ON', 'BBWI', 'MOS', 'DOW', 'HPQ', 'JNPR', 'IP', 'KMI', 'WM', 'XLE', 'FI', 'URI', 'DD', 'APA', 'HRL', 'T', 'MDT', 'DLR', 'DISH', 'NSC', 'HES', 'MDLZ', 'PXD', 'ISRG', 'KLAC', 'PSX', 'SPGI', 'ABC', 'LRCX', 'CI', 'WDC', 'DHI', 'ENPH', 'LQD', 'CHTR', 'EXPE', 'TMUS', 'ACN', 'CF', 'VLO', 'NEM', 'CME', 'FOXA', 'TLT', 'GILD', 'ULTA', 'CMCSA', 'PM', 'PARA', 'WHR', 'CAG', 'SO', 'AMGN', 'OXY', 'TFC', 'ANET', 'CPB', 'MO', 'BBY', 'NTAP', 'GNRC', 'NUE', 'WBA', 'CAH', 'GLD', 'HBAN', 'MAR', 'TPR', 'DLTR', 'AFL', 'UNH', 'GEHC', 'SEDG', 'UNP', 'ALB', 'CBOE', 'DPZ', 'XLK', 'SWKS', 'USB', 'EFA

In [28]:
s3 = boto3.client('s3')

idx = ["QQQ","SPY","IWM"]

def run_process(date_str):
    # try:
    raw = build_pcr_features(date_str)
    # except Exception as e:
    #     print(f"{date_str} {e}")
    #     build_pcr_features(date_str)
    return raw


def generate_dates_historic_vol(date_str):
    end = datetime.strptime(date_str, "%Y-%m-%d")
    start = end - timedelta(weeks=4)
    to_stamp = end.strftime("%Y-%m-%d")
    hour_stamp = end.strftime("%Y-%m-%d")
    from_stamp = start.strftime("%Y-%m-%d")
    return from_stamp, to_stamp, hour_stamp

def build_pcr_features(date_str):
    hours = ["10","11","12","13","14","15"]
    for hour in hours:
        key_str = date_str.replace("-","/")
        s3 = get_s3_client()
        from_stamp, to_stamp, hour_stamp = generate_dates_historic_vol(date_str)
        raw_pcr_data = pull_pcr_data(from_stamp,to_stamp,hours, current_hour=hour)
        pcr_df = pcr_feature_engineering(raw_pcr_data)
        put_response = s3.put_object(Bucket="inv-alerts", Key=f"idx_alerts/{key_str}/{hour}/pcr_features.csv", Body=pcr_df.to_csv())

    # for hour in hours:
    #     df = s3.get_object(Bucket="inv-alerts", Key=f"all_alerts/{key_str}/{hour}.csv")
    #     df = pd.read_csv(df['Body'])
    #     idx = df.loc[df['symbol'].isin(idx)]
    #     raw_pcr_data = pull_pcr_data(from_stamp,to_stamp,hour)
    #     # pcr_df = pcr_feature_engineering(idx,raw_pcr_data)
    #     # put_response = s3.put_object(Bucket="inv-alerts", Key=f"idx_alerts/{key_str}/{hour}.csv", Body=pcr_df.to_csv())
    return raw_pcr_data


def pull_pcr_data(from_stamp,to_stamp,hours,current_hour):
    date_list = build_date_list(from_stamp,to_stamp)
    raw_pcr_data = {}
    for symbol in idx:
        dfs = []
        for date_str in date_list:
            for hour in hours:
                if date_str == date_list[-1] and hour > current_hour:
                    continue
                else:
                    key_str = date_str.replace("-","/")
                    try:
                        df = s3.get_object(Bucket="icarus-research-data", Key=f"options_snapshot/{key_str}/{hour}/{symbol}.csv")
                        df = pd.read_csv(df['Body'])
                        df['date'] = key_str
                        df['date_hour'] = f"{key_str}-{hour}"
                    except Exception as e:
                        print(f"options_snapshot/{key_str}/{hour}/{symbol}.csv {e}")
                        continue
                    dfs.append(df)
        full_sym = pd.concat(dfs)
        raw_pcr_data[symbol] = full_sym
    return raw_pcr_data

def build_date_list(from_stamp,to_stamp):
    start_date = datetime.strptime(from_stamp, "%Y-%m-%d")
    end_date = datetime.strptime(to_stamp, "%Y-%m-%d")
    date_diff = end_date - start_date
    numdays = date_diff.days 
    date_list = []
    print(numdays)
    for x in range (0, numdays):
        temp_date = start_date + timedelta(days = x)
        if temp_date.weekday() < 5:
            date_str = temp_date.strftime("%Y-%m-%d")
            date_list.append(date_str)

    return date_list


def pcr_feature_engineering(raw_pcr_data):
    feature_data = {}
    for symbol in idx:
        sym_data = raw_pcr_data[symbol]
        aggregated_df = sym_data.groupby(['date_hour', 'option_type'])['volume'].sum().reset_index()
        pivot_df = aggregated_df.pivot(index='date_hour', columns='option_type', values='volume')
        pivot_df.columns = ['call_volume', 'put_volume']
        pivot_df['total_volume'] =  pivot_df['call_volume'] + pivot_df['put_volume']
        hour = sym_data.iloc[-1]
        day = sym_data.iloc[-6:]
        ten_day = sym_data.iloc[-60:]
        hour_pcr = hour['put_volume'].sum() / hour['call_volume'].sum()
        day_pcr = day['put_volume'].sum() / day['call_volume'].sum()
        ten_day_pcr = ten_day['put_volume'].sum() / ten_day['call_volume'].sum()
        hour_pcr_dff1D = ((hour_pcr - day_pcr)/ day_pcr) * 100
        hour_pcr_dff10D = ((hour_pcr - ten_day_pcr)/ ten_day_pcr) * 100
        day_pcr_dff10D = ((day_pcr - ten_day_pcr)/ ten_day_pcr) * 100
        feature_data[symbol] = [hour_pcr, day_pcr, ten_day_pcr, hour_pcr_dff1D, hour_pcr_dff10D, day_pcr_dff10D]
    feature_df = pd.DataFrame.from_dict(feature_data, orient='index', columns=['hour_pcr', 'day_pcr', 'ten_day_pcr', 'hour_pcr_dff1D', 'hour_pcr_dff10D', 'day_pcr_dff10D'])
    return feature_df


In [29]:
result = run_process("2021-03-05")

28
options_snapshot/2021/02/15/10/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/11/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/12/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/13/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/14/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/15/QQQ.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_snapshot/2021/02/15/10/SPY.csv An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
options_sn

In [34]:
qqq = result["QQQ"]
qqq

,Unnamed: 0,high,low,volume,symbol,option_type,date,date_hour
0,0,60.00,60.00,1,O:QQQ210205C00270000,call,2021/02/05,2021/02/05-10
1,1,55.47,55.47,5,O:QQQ210205C00275000,call,2021/02/05,2021/02/05-10
2,2,52.78,52.78,1,O:QQQ210205C00277500,call,2021/02/05,2021/02/05-10
3,3,NaN,NaN,0,O:QQQ210212C00277500,call,2021/02/05,2021/02/05-10
4,4,NaN,NaN,0,O:QQQ210212C00278000,call,2021/02/05,2021/02/05-10
...,...,...,...,...,...,...,...,...
340,340,46.10,46.10,2,O:QQQ210312P00350000,put,2021/03/04,2021/03/04-15
341,341,49.78,49.78,10,O:QQQ210312P00360000,put,2021/03/04,2021/03/04-15
342,342,51.45,51.45,10,O:QQQ210312P00362500,put,2021/03/04,2021/03/04-15
343,343,61.59,61.50,18,O:QQQ210312P00365000,put,2021/03/04,2021/03/04-15


In [38]:
aggregated_df = qqq.groupby(['date_hour', 'option_type'])['volume'].sum().reset_index()

# Create a pivot table
pivot_df = aggregated_df.pivot(index='date_hour', columns='option_type', values='volume')

# Rename the columns for clarity
pivot_df.columns = [f"{col}_volume" for col in pivot_df.columns]
pivot_df['total_volume'] =  pivot_df['call_volume'] + pivot_df['put_volume']

print(pivot_df)

               call_volume  put_volume  total_volume
date_hour                                           
2021/02/05-10        51809       85370        137179
2021/02/05-11        76749      111139        187888
2021/02/05-12       120313      144757        265070
2021/02/05-13       146522      172695        319217
2021/02/05-14       168513      195684        364197
...                    ...         ...           ...
2021/03/04-11       249227      399602        648829
2021/03/04-12       357546      541605        899151
2021/03/04-13       470040      708422       1178462
2021/03/04-14       560567      815306       1375873
2021/03/04-15       623709      945539       1569248

[114 rows x 3 columns]
